# ETL 06 - Scripts Area Dimensional SQL
    Objetivo: Gerar arquivo CSV mais completo para os cientistas de dados
    Desenvolvedor: Marcelo Muniz de Alencar

In [2]:
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=PCMARCELO\\SQLEXPRESS;'
                      'Database=DDTM_OiMasterDados;'
                      'Trusted_Connection=yes;')

In [3]:
import pandas as pd
sql_query = pd.read_sql_query('''
                              declare @MediaTemperatura as float
declare @DesvioPadraoTemperatura as float

set @MediaTemperatura = 
(select  AVG(ROUND(([ValMedioTemperaturaC]),2))
from [DDTM_OiMasterDados].[OiMasterDados].[TOiFAT_Temperatura] (NOLOCK))

set @DesvioPadraoTemperatura = 
(select  STDEVP(ROUND(([ValMedioTemperaturaC]),2))
from [DDTM_OiMasterDados].[OiMasterDados].[TOiFAT_Temperatura] (NOLOCK))

SELECT 
	   T1.[CodSkData]					as DimCalendario_CodSkData
      ,T3.CodBkData						as DimCalendario_DataCalendario
	  ,T3.NumAno						as DimCalendario_NumAno
	  ,T3.DescAno						as DimCalendario_DescAno
	  ,T3.NumSemestre					as DimCalendario_NumSemestre
	  ,T3.DescSemestre					as DimCalendario_DescSesmestre
	  ,T3.NumTrimestre					as DimCalendario_NumTrimestre
	  ,T3.DescTrimestre					as DimCalendario_DescTrimestre
	  ,T3.NumMes						as DimCalendario_NumMes
	  ,T3.DescMes						as DimCalendario_DescMes
	  ,T3.NumSemanaAno					as DimCalendario_NumSemanaAno
	  ,T3.DescSemanaAno					as DimCalendario_DescSemanaAno
	  ,T3.NumDia						as DimCalendario_NumDia
	  ,T3.NumDiaSemana					as DimCalendario_NumDiaSemana
	  ,case when T3.NumDiaSemana = 1 then 'DOM'
			when T3.NumDiaSemana = 2 then 'SEG'
			when T3.NumDiaSemana = 3 then 'TER'
			when T3.NumDiaSemana = 4 then 'QUA'
			when T3.NumDiaSemana = 5 then 'QUI'
			when T3.NumDiaSemana = 6 then 'SEX'
			else 'SAB'
	   end								as DimCalendario_DescDiaSemana
	  ,T3.FlgFinalSemana				as DimCalendario_FlgFinalSemana
	  ,CASE WHEN T2.FlgHemisferioNorte = 'S'
				THEN T3.DescEstacaoBoreal
	  			ELSE T3.DescEstacaoAustral
	   END								as DimCalendario_DescEstacao
      ,T1.[CodSkCidade]					as DimCidade_CodSkCidade
	  ,T2.NomCidade						as DimCidade_NomCidade
	  ,T2.NomEstado						as DimCidade_NomEstado
	  ,T2.NomPais						as DimCidade_NomPais
	  ,T2.NomRegiaoGlobo				as DimCidade_NomRegiaoGlobo
	  ,T2.ValLatitude					as DimCidade_ValLatitude
	  ,T2.ValLongitude					as DimCidade_ValLongitude
	  ,T2.FlgHemisferioNorte			as DimCidade_FlgHemisferioNorte
	  ,T1.[ValMedioTemperaturaC]		as FatTempearatura_ValMedioTemperaturaCelsius
	  ,T2.CategoriaInverno				as ClusterCidade_CategoriaInverno
	  ,T2.TempMediaInverno				as ClusterCidade_TempMediaInverno
      ,T2.CategoriaOutono				as ClusterCidade_CategoriaOutono
	  ,T2.TempMediaOutuno				as ClusterCidade_TempMediaOutuno
	  ,T2.CategoriaPrimavera			as ClusterCidade_CategoriaPrimavera
	  ,T2.TempMediaPrimavera			as ClusterCidade_TempMediaPrimavera
      ,T2.CategoriaVerao				as ClusterCidade_CategoriaVerao
	  ,T2.TempMediaVerao				as ClusterCidade_TempMediaVerao
	  ,T2.ClusterDefinicao 				as ClusterCidade_ClusterDefinicao
	  ,T2.ClusterFinal 					as ClusterCidade_ClusterFinal
FROM DDTM_OiMasterDados.[OiMasterDados].[TOiFAT_Temperatura] AS T1 (NOLOCK)
		INNER JOIN
	 (
	 SELECT Cidade.[CodSkCidade]
      ,Cidade.[NomCidade]
      ,Cidade.[NomEstado]
      ,Cidade.[NomPais]
      ,Cidade.[NomRegiaoGlobo]
      ,Cidade.[ValLatitude]
      ,Cidade.[ValLongitude]
      ,Cidade.[FlgHemisferioNorte]
	  ,case when [Categoria_Inverno1de3] = 1
				then 'INVERNO 1'
			when [Categoria_Inverno2de3] = 1
				then 'INVERNO 2'
				else 'INVERNO 3' ---[Categoria_Inverno3de3]
	   end as CategoriaInverno
	  ,inverno.MediaCidade as TempMediaInverno
      ,case when [Categoria_Outono1de3] = 1
				then 'OUTONO 1'
			when [Categoria_Outono2de3] = 1
				then 'OUTONO 2'
				else 'OUTONO 3' ---[Categoria_Inverno3de3]
	   end as CategoriaOutono
	  ,outono.MediaCidade as TempMediaOutuno
	  ,case when [Categoria_Primavera2de3] = 1
				then 'PRIMAVERA 2'
			when [Categoria_Primavera3de3] = 1
				then 'PRIMAVERA 3'
				else 'PRIMAVERA 1' ---[Categoria_Inverno3de3]
	   end as CategoriaPrimavera
	  ,primavera.MediaCidade as TempMediaPrimavera
      ,case when [Categoria_Verão1de3] = 1
				then 'VERÃO 1'
			when [Categoria_Verão2de3] = 1
				then 'VERÃO 2'
				else 'VERÃO 3' ---[Categoria_Inverno3de3]
	   end as CategoriaVerao
	  ,verao.MediaCidade as TempMediaVerao
	  ,'... onde 1 mais frio, 3 mais quente' as ClusterDefinicao
	  ,Cluster.Cluster as ClusterFinal
FROM [DDTM_OiMasterDados].[OiMasterDados].[TOiDIM_Cidade] as Cidade (NOLOCK)
		INNER JOIN
	 [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_BaseClusterCidade3R] as Cluster (NOLOCK)
		ON Cidade.CodSkCidade = Cluster.CodSkCidade
		INNER JOIN
	[DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_BaseClusterCidade2R] as DetCluster (NOLOCK)
		ON Cidade.CodSkCidade = DetCluster.CodSkCidade
		INNER JOIN
	 (SELECT CodSkCidade, [MediaCidade]
	  FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_BaseClusterCidadeR] (NOLOCK)
      WHERE Categoria like '%inverno%') as inverno
		ON Cidade.CodSkCidade = inverno.CodSkCidade
		INNER JOIN
	 (SELECT CodSkCidade, [MediaCidade]
	  FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_BaseClusterCidadeR] (NOLOCK)
      WHERE Categoria like '%outono%') as outono
		ON Cidade.CodSkCidade = outono.CodSkCidade
		INNER JOIN
	 (SELECT CodSkCidade, [MediaCidade]
	  FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_BaseClusterCidadeR] (NOLOCK)
      WHERE Categoria like '%verão%') as verao
		ON Cidade.CodSkCidade = verao.CodSkCidade
		INNER JOIN
	 (SELECT CodSkCidade, [MediaCidade]
	  FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_BaseClusterCidadeR] (NOLOCK)
      WHERE Categoria like '%primavera%') as primavera
		ON Cidade.CodSkCidade = primavera.CodSkCidade
	 )AS T2 
		ON T1.CodSkCidade = T2.CodSkCidade
		INNER JOIN
	 DDTM_OiMasterDados.[OiMasterDados].[TOiDIM_Calendario] AS T3 (NOLOCK)
		ON T1.CodSkData = T3.CodSkData
--- retira os outliers 
where (t1.ValMedioTemperaturaC-@MediaTemperatura)/@DesvioPadraoTemperatura > -3
and (t1.ValMedioTemperaturaC-@MediaTemperatura)/@DesvioPadraoTemperatura < 3
ORDER BY T2.CodSkCidade, T3.CodSkData
                              '''
                              ,conn) 
df = pd.DataFrame(sql_query)
df.to_csv (r'C:\Users\MarceloMuniz\OneDrive\# Oi Master Dados - 3ª Fase\_TrabalhoFinal\docsDrive\CityTemperatureAjustada.csv', index = False) 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [7]:
df.shape

(2845082, 36)